### File for predicting the results of the trained model

Author :  Vansh Goenka

#Imports

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
# Download the stop words corpus if not already downloaded
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import re
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, f1_score
import os
# os.environ["KERAS_BACKEND"] = "tensorflow"
%env TF_USE_LEGACY_KERAS=1
import tensorflow as tf
import transformers
import numpy as np






[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


env: TF_USE_LEGACY_KERAS=1


#Setting up the dataframes and model

In [ ]:
# Load the training data from CSV
from google.colab import drive
drive.mount('/content/drive')

trial_data = pd.read_csv('/content/drive/My Drive/ED_trial.csv')
train_data = pd.read_csv('/content/drive/My Drive/train.csv')
dev_data = pd.read_csv('/content/drive/My Drive/dev.csv')
eval_data = pd.read_csv('/content/drive/My Drive/eval.csv')
dev_data['Evidence'] = dev_data['Evidence'].astype(str)

Mounted at /content/drive


#Pre-processing the data

In [ ]:
import transformers
model_roberta = tf.keras.models.load_model('/content//drive/My Drive/RoBERTa_model.h5', custom_objects={"TFRobertaModel": transformers.TFRobertaModel})

##Function to evluate the results using the same metrics as EvalAI- check if its needed

In [ ]:

# Evaluation function to measure model performance
def evaluate(actual, predicted):
    accuracy = accuracy_score(actual, predicted)
    print(f"Accuracy: {accuracy}")
    report = classification_report(actual, predicted)
    print(f"Classification report: {report}")
    f1_macro = f1_score(actual, predicted, average='macro')
    print(f"F1 score - macro: {f1_macro}")
    precision_macro = precision_score(actual, predicted, average='macro')
    print(f"Precision - macro: {precision_macro}")
    recall_macro = recall_score(actual, predicted, average='macro')
    print(f"Recall - macro: {recall_macro}")
    f1_weighted = f1_score(actual, predicted, average='weighted')
    print(f"F1 score (weighted): {f1_weighted}")
    precision_weighted = precision_score(actual, predicted, average='weighted')
    print(f"Precision (weighted): {precision_weighted}")
    recall_weighted = recall_score(actual, predicted, average='weighted')
    print(f"Recall (weighted): {recall_weighted}")
    labels = [0, 1]
    cm = confusion_matrix(actual, predicted)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, cmap="Blues", fmt='g', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

#Loading the models

#Using RoBERTa

In [ ]:


# # Create a new folder called "Models" in /content/resources/
# !mkdir -p /content/resources/Models

# # Download the models from google drive into the "Models" folder
# !gdown --output "/content/resources/Models/t5_model.h5" 1cF2SIFknn-DVoYo2xjKP4raSmLAlyvIU
# !gdown --output "/content/resources/Models/t5_flan_model.h5" 1-D7BhUqyX0kDixryv6WbDv5HX2kPnbbs
# !gdown --output "/content/resources/Models/roberta_model.h5" 1-L4AsV0kLzZ49DWK9bYq4OTslVc8y1_P

load models using transformers

In [ ]:

# #Load models as transformers
# modelT5 = tf.keras.models.load_model('/content/resources/Models/t5_model.h5', custom_objects={"TFT5EncoderModel": transformers.TFT5EncoderModel})
# modelRoBERTa = tf.keras.models.load_model('/content/resources/Models/roberta_model.h5', custom_objects={"TFRobertaModel": transformers.TFRobertaModel})
# modelFlanT5 = tf.keras.models.load_model('/content/resources/Models/t5_flan_model.h5', custom_objects={"TFT5EncoderModel": transformers.TFT5EncoderModel})


Writing predicted labels to csv
Generating Dataframe

In [ ]:

# pd.set_option('display.max_rows', None)
# result_df = pd.DataFrame({'prediction': ensemble_predictions})
# column_name_row = pd.DataFrame({'prediction': ['prediction']}, index=[0])
# result_df['prediction'] = result_df['prediction'].astype(int)
# result_df = pd.concat([column_name_row, result_df]).reset_index(drop=True)
# result_df


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


NameError: name 'choice' is not defined

In [ ]:

# import numpy as np
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

def roberta_encode(claims, evidences, tokenizer, max_length=120):
    concatenated_inputs = [c + '  ' + e for c, e in zip(np.array(evidences), np.array(claims))]

    inputs = tokenizer(
        concatenated_inputs,
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='tf'
    )

    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask']
    }

train_input_Roberta = roberta_encode(train_data['Claim'].values, train_data['Evidence'].values, tokenizer)
dev_input_Roberta = roberta_encode(dev_data['Claim'].values, dev_data['Evidence'].values, tokenizer)
trial_input_Roberta = roberta_encode(trial_data['Claim'].values, trial_data['Evidence'].values, tokenizer)







In [ ]:
predictionsRoBERTa = np.argmax(model_roberta.predict(trial_input_Roberta), axis=1)

2/2 [==============================] - 28s 7s/step


In [ ]:
# Calculate accuracy
acc = np.mean(predictionsRoBERTa == trial_data.label.values)
print("Accuracy:", acc)

Accuracy: 0.96
